In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import re



We found an API option, which would be easier then scraping but did the scraping anyway 

https://pypi.org/project/weathercom/



In [4]:
# (chrome) Webscraper plugin identifies elements to apply to Pandas function, identical to Chrome's Developer Tools


Here are the result from the Chrome Plugin 

In [6]:
df = pd.read_csv('./result.csv')
df


FileNotFoundError: [Errno 2] No such file or directory: './result.csv'

In [2]:
## Experiment to shrink code and manipulating data frames Ignore this the cell below for peer review!!!

In [7]:


# get the web page
page = requests.get("https://weather.com/weather/tenday/l/69bedc6a5b6e977993fb3e5344e3c06d8bc36a1fb6754c3ddfb5310a3c6d6c87")

#parse html
soup = BeautifulSoup(page.content, 'html.parser')

#Grab the varibales we need 
dailysummary = soup.find_all(class_="DailyContent--narrative--3AcXd")
temp = soup.find_all(class_="DailyContent--temp--_8DL5")
date = soup.find_all(class_="DailyContent--daypartDate--3MM0J")

#function to strip tags 
def myStrip(i):
    a = 0
    for a in range(0, len(i) ,1):
       i[a] = i[a].text
    #print(i)

myStrip(date)
myStrip(temp)
myStrip(dailysummary)

beginFrame = {'Date':date, 'Temp':temp, 'DailySummary':dailysummary}
frame = pd.DataFrame(beginFrame)


frame2 = frame.drop_duplicates('Date').reset_index().rename(columns={'Temp':'highTemp', 'DailySummary':'Day_Summary'})

frame3 = frame.drop_duplicates('Date',keep='last').reset_index().rename(columns={'Temp':"lowTemp", 'DailySummary' : 'Night_Summary'})

frame = frame2.merge(frame3,how='inner', on=['Date'])
frame[['Day','Date']] = frame.Date.str.split(" ",expand=True,)
frame = frame[["Day","Date","highTemp","lowTemp","Day_Summary","Night_Summary"]]

frame

,Day,Date,highTemp,lowTemp,Day_Summary,Night_Summary
0,Thu,10,45°,45°,Cloudy. Low near 45F. Winds WNW at 10 to 20 mph.,Cloudy. Low near 45F. Winds WNW at 10 to 20 mph.
1,Fri,11,55°,48°,Mainly cloudy. High near 55F. Winds W at 10 to...,Rain. Low 48F. Winds SW at 10 to 20 mph. Chanc...
2,Sat,12,58°,49°,"Showers in the morning, then cloudy in the aft...",Cloudy with occasional light rain after midnig...
3,Sun,13,57°,46°,Rain early...then remaining cloudy with shower...,Partly to mostly cloudy. Low 46F. Winds WNW at...
4,Mon,14,57°,44°,Partly cloudy skies. High 57F. Winds NW at 5 t...,Partly cloudy during the evening followed by c...
5,Tue,15,56°,46°,Partly cloudy skies in the morning will give w...,Considerable cloudiness. Low 46F. Winds light ...
6,Wed,16,58°,46°,Mostly cloudy skies early will become partly c...,Cloudy with occasional showers. Low 46F. Winds...
7,Thu,17,57°,44°,Intervals of clouds and sunshine. High 57F. Wi...,Partly cloudy skies early will become overcast...
8,Fri,18,58°,45°,Partly cloudy. High 58F. Winds N at 5 to 10 mph.,Partly cloudy skies during the evening will gi...
9,Sat,19,58°,46°,Cloudy with a few showers. High 58F. Winds SSW...,Overcast with rain showers at times. Low 46F. ...


In [34]:
# -*- coding: utf-8 -*-
# Tidy up data
dateClean = []
dayClean = []
tempClean = []
sumClean = []
dn = []

for d in date:
    dd = d.get_text()
    ddd = dd.split(" ")
    dateClean.append(ddd[1])
    dayClean.append(ddd[0])

for t in temp:
    tst = t.get_text()
    tt = re.sub('[^0-9]', '', tst)
    # T(°C) = (T(°F) - 32) × 5/9
    tc = (int(tt)-32) * 5/9
    tempClean.append(tc)

for s in dailysummary:
    ss = s.get_text()
    sumClean.append(ss)

for i in range(0,30):
    if i % 2 !=0:
        dn.append("d")
    else:
        dn.append("n")  

  
data_dict = {"day": dayClean, "date": dateClean, "temp": tempClean, 'Summary': sumClean, "day/night":dn}
#print(dateClean)
#print(dayClean)
#print(tempClean)
print(len(sumClean))
print(data_dict)

30
{'day': ['Wed', 'Wed', 'Thu', 'Thu', 'Fri', 'Fri', 'Sat', 'Sat', 'Sun', 'Sun', 'Mon', 'Mon', 'Tue', 'Tue', 'Wed', 'Wed', 'Thu', 'Thu', 'Fri', 'Fri', 'Sat', 'Sat', 'Sun', 'Sun', 'Mon', 'Mon', 'Tue', 'Tue', 'Wed', 'Wed'], 'date': ['09', '09', '10', '10', '11', '11', '12', '12', '13', '13', '14', '14', '15', '15', '16', '16', '17', '17', '18', '18', '19', '19', '20', '20', '21', '21', '22', '22', '23', '23'], 'temp': [17.22222222222222, 6.666666666666667, 13.88888888888889, 6.666666666666667, 13.333333333333334, 7.222222222222222, 13.333333333333334, 8.333333333333334, 14.444444444444445, 8.333333333333334, 14.444444444444445, 6.666666666666667, 13.88888888888889, 6.666666666666667, 14.444444444444445, 7.222222222222222, 14.444444444444445, 7.222222222222222, 14.444444444444445, 6.666666666666667, 15.0, 6.666666666666667, 15.555555555555555, 6.111111111111111, 15.555555555555555, 5.555555555555555, 15.555555555555555, 5.555555555555555, 15.555555555555555, 5.555555555555555], 'Summary'

Here we decided to try 2 fdifferent approachs, the first uses pyhton to manipulate the data, the second uses pandas to manipulate the data. 


In [152]:
#Make changes to data before pandas 
nDay = dayClean[::2]
nDate = dateClean[::2]
high = []
low = []
new_sum = []

for te in tempClean:
    ind = tempClean.index(te)
    if ind % 2 != 0:
        ns = "Day: " + sumClean[ind] + "Night: " + sumClean[ind+1]
        high.append(te)
        new_sum.append(ns)
    else: 
        low.append(te)

new_data_dict = {"date":nDate, "day":nDay, "summary":new_sum, "highTemp":high, "lowTemp": low}
w_dn = pd.DataFrame(new_data_dict)
w_dn

,date,day,summary,highTemp,lowTemp
0,09,Wed,Day: Partly cloudy this evening with more clou...,6.666667,17.222222
1,10,Thu,Day: Partly cloudy this evening with more clou...,6.666667,13.888889
2,11,Fri,Day: Mostly cloudy in the evening then periods...,7.222222,13.333333
3,12,Sat,Day: Cloudy with occasional showers late at ni...,8.333333,13.333333
4,13,Sun,Day: Cloudy with occasional showers late at ni...,8.333333,14.444444
5,14,Mon,Day: Partly cloudy this evening with more clou...,6.666667,14.444444
6,15,Tue,Day: Partly cloudy this evening with more clou...,6.666667,13.888889
7,16,Wed,Day: Mostly cloudy in the evening then periods...,7.222222,14.444444
8,17,Thu,Day: Mostly cloudy in the evening then periods...,7.222222,14.444444
9,18,Fri,Day: Partly cloudy this evening with more clou...,6.666667,14.444444


In [76]:
#Making the mat plot lib 
# this only works in Juypter note book not deepnote 
w_dn['date'] = w_dn['date'].astype(float)
w_dn.plot()
plt.show()

## Second approach 

In [132]:
weather1  = pd.DataFrame(data_dict)
weather2 = weather1
weather3 = weather1
#print(weather2.drop_duplicates('date'))
#print(weather3.drop_duplicates('date',keep='last'))
weather2 = weather2.drop_duplicates('date')
weather3 = weather3.drop_duplicates('date',keep='last')

weather2 = weather2.reset_index()
weather3 = weather3.reset_index()
weather2 = weather2.rename(columns={'temp':'highTemp', 'Summary':'Day_Summary'})
weather3 = weather3.rename(columns={'temp':"lowTemp", 'Summary' : 'Night_Summary'})


Comdf = weather2.merge(weather3,how='inner', on=['day','date'])

finalframe = Comdf[["day","date","highTemp","lowTemp","Day_Summary","Night_Summary"]]
finalframe


#thanks

#weather2.

,day,date,highTemp,lowTemp,Day_Summary,Night_Summary
0,Wed,09,17.222222,6.666667,A mix of clouds and sun. High 63F. Winds N at ...,Partly cloudy this evening with more clouds fo...
1,Thu,10,13.888889,6.666667,Some clouds in the morning will give way to ma...,Mostly clear evening skies will give way to mo...
2,Fri,11,13.333333,7.222222,Partly cloudy. High 56F. Winds NW at 5 to 10 mph.,Mostly cloudy in the evening then periods of s...
3,Sat,12,13.333333,8.333333,Cloudy with occasional rain showers. High 56F....,Cloudy with occasional showers late at night. ...
4,Sun,13,14.444444,8.333333,A steady rain in the morning. Showers continui...,Overcast with rain showers at times. Low 47F. ...
5,Mon,14,14.444444,6.666667,Partly cloudy skies. High 58F. Winds WNW at 10...,Mostly clear during the evening followed by mo...
6,Tue,15,13.888889,6.666667,Sunshine and clouds mixed. High 57F. Winds W a...,Partly cloudy skies early followed by increasi...
7,Wed,16,14.444444,7.222222,Considerable cloudiness with occasional rain s...,Cloudy with occasional rain showers. Low near ...
8,Thu,17,14.444444,7.222222,Rain showers in the morning becoming more inte...,Considerable cloudiness. Low around 45F. Winds...
9,Fri,18,14.444444,6.666667,A few showers early with mostly cloudy conditi...,Overcast with rain showers at times. Low 44F. ...


##Sergey Brin

Each "h2" element as a column of you final pandas dataframe

What can you scrape that makes sense? Think about creating a dataset. 
Places, dates, content. Some values will be NaN.

In [143]:
sg_site = requests.get('http://infolab.stanford.edu/~sergey/resume.html')
sg_soup = BeautifulSoup(sg_site.content, 'html.parser')

h_2_tags = sg_soup.find_all('h2')
cols = []
for h in h_2_tags:
    cols.append(h.get_text())

#print(sg_soup.prettify())

##Some Other Websites - Football transfers 

Scraping bonus task 

In [151]:
f_pages = requests.get ('https://www.ox.ac.uk/admissions/undergraduate/courses/course-listing')
f_soup = BeautifulSoup(f_pages.content, 'html.parser')
f_body = f_soup.find("body")
#tbc 

This is the 2 line version of the approach above 

In [155]:
# the one line version 
weather1  = pd.DataFrame(data_dict)
weather2 = weather1
weather3 = weather1

weather2 = weather2.drop_duplicates('date').reset_index().rename(columns={'temp':'highTemp', 'Summary':'Day_Summary'})
weather3 = weather3.drop_duplicates('date',keep='last').reset_index().rename(columns={'temp':"lowTemp", 'Summary' : 'Night_Summary'})

Comdf = weather2.merge(weather3,how='inner', on=['day','date'])

finalframe = Comdf[["day","date","highTemp","lowTemp","Day_Summary","Night_Summary"]]
finalframe['highTemp'] = finalframe['highTemp'].round(decimals=2)

finalframe



/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,day,date,highTemp,lowTemp,Day_Summary,Night_Summary
0,Wed,09,17.22,6.666667,A mix of clouds and sun. High 63F. Winds N at ...,Partly cloudy this evening with more clouds fo...
1,Thu,10,13.89,6.666667,Some clouds in the morning will give way to ma...,Mostly clear evening skies will give way to mo...
2,Fri,11,13.33,7.222222,Partly cloudy. High 56F. Winds NW at 5 to 10 mph.,Mostly cloudy in the evening then periods of s...
3,Sat,12,13.33,8.333333,Cloudy with occasional rain showers. High 56F....,Cloudy with occasional showers late at night. ...
4,Sun,13,14.44,8.333333,A steady rain in the morning. Showers continui...,Overcast with rain showers at times. Low 47F. ...
5,Mon,14,14.44,6.666667,Partly cloudy skies. High 58F. Winds WNW at 10...,Mostly clear during the evening followed by mo...
6,Tue,15,13.89,6.666667,Sunshine and clouds mixed. High 57F. Winds W a...,Partly cloudy skies early followed by increasi...
7,Wed,16,14.44,7.222222,Considerable cloudiness with occasional rain s...,Cloudy with occasional rain showers. Low near ...
8,Thu,17,14.44,7.222222,Rain showers in the morning becoming more inte...,Considerable cloudiness. Low around 45F. Winds...
9,Fri,18,14.44,6.666667,A few showers early with mostly cloudy conditi...,Overcast with rain showers at times. Low 44F. ...
